In [16]:
import torch.nn as nn
import torch
import pytorch_lightning as pl
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score

In [17]:
df = pd.read_csv("loan_data.csv")
for x in df.columns:
    if "." in (x):
        df.rename(columns={x:x.replace(".","_")} , inplace = True)
df[df["not_fully_paid"] == 0]

,credit_policy,purpose,int_rate,installment,log_annual_inc,dti,fico,days_with_cr_line,revol_bal,revol_util,inq_last_6mths,delinq_2yrs,pub_rec,not_fully_paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9561,0,all_other,0.0788,115.74,10.999095,10.17,722,4410.000000,11586,61.6,4,0,0,0
9562,0,debt_consolidation,0.1348,508.87,10.933107,17.76,717,3870.041667,8760,28.2,6,0,0,0
9564,0,all_other,0.1385,511.56,12.323856,12.33,687,6420.041667,385489,51.2,4,0,0,0
9567,0,all_other,0.1311,101.24,10.968198,8.23,687,2790.041667,1514,13.8,5,0,0,0


In [18]:
categorical_columns = ['purpose']
numerical_columns = ['int_rate', 'installment', 'log_annual_inc',
       'dti', 'fico', 'days_with_cr_line', 'revol_bal', 'revol_util',
       'inq_last_6mths', 'delinq_2yrs', 'pub_rec']
label = "not_fully_paid"

In [19]:
df1 = df[df["not_fully_paid"] == 1].sample(1533)
df2 = df[df["not_fully_paid"] == 1].sample(1533)
df3 = df[df["not_fully_paid"] == 1].sample(1533)
df4 = df[df["not_fully_paid"] == 1].sample(1533)
df5 = df[df["not_fully_paid"] == 1].sample(1533)
df6 = df[df["not_fully_paid"] == 1].sample(1533)
df7 = df[df["not_fully_paid"] == 1].sample(1533)
df8 = df[df["not_fully_paid"] == 1].sample(1533)
dff = pd.concat([df,df1,df2,df3,df4,df5,df6,df7,df8] ,ignore_index = True)
dff

,credit_policy,purpose,int_rate,installment,log_annual_inc,dti,fico,days_with_cr_line,revol_bal,revol_util,inq_last_6mths,delinq_2yrs,pub_rec,not_fully_paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21837,1,all_other,0.1189,50.58,11.500205,8.12,677,3359.958333,36192,84.6,0,0,0,1
21838,1,credit_card,0.1241,160.38,11.050890,13.81,672,2880.000000,5034,45.4,3,1,1,1
21839,1,debt_consolidation,0.0894,158.86,10.961278,14.33,727,7020.041667,2530,5.2,0,1,0,1
21840,1,debt_consolidation,0.0800,188.02,11.429587,5.07,752,8162.958333,21365,39.4,0,0,0,1


In [20]:
X = dff[numerical_columns]
y = dff[[label]]

In [21]:
train_x,test_x,train_y,test_y = train_test_split(X,y,test_size = 0.5,random_state = 42, stratify = y[label])
test_x,val_x,test_y,val_y = train_test_split(test_x,test_y,test_size = 0.5,random_state = 42 , stratify = test_y[label])

In [22]:
val_x

,int_rate,installment,log_annual_inc,dti,fico,days_with_cr_line,revol_bal,revol_util,inq_last_6mths,delinq_2yrs,pub_rec
8725,0.1159,231.14,11.066763,10.63,687,3149.958333,5260,24.2,10,0,0
6992,0.1287,403.60,10.553090,14.98,722,1560.041667,7060,71.8,0,0,0
10856,0.1083,458.85,12.100712,6.24,747,3600.000000,60936,24.7,2,0,0
1508,0.1272,293.65,11.369171,20.12,682,2790.000000,25965,52.5,1,0,0
12001,0.1028,150.66,10.727663,3.26,687,6509.958333,105,35.0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
14029,0.1253,334.67,10.596635,20.46,702,2520.000000,7722,66.1,0,0,0
4682,0.1253,430.88,10.539006,24.27,712,2039.958333,5048,55.5,0,0,0
16266,0.0894,127.09,11.082143,12.24,742,11190.958330,8782,38.3,4,0,0
10609,0.1774,720.42,11.314475,2.87,677,4290.958333,9603,51.9,0,1,0


In [23]:
class LoanDataset(Dataset):
    def __init__(self,features,labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        feature = self.features[index]
        label = self.labels[index]
        return torch.tensor(feature).float() , torch.tensor(label).float()

In [24]:
loans = LoanDataset(train_x.values , train_y.values)
features , labels = next(iter(DataLoader(loans,batch_size = 1 , shuffle = True)))
print(features,labels)

tensor([[8.5900e-02, 7.2710e+01, 1.0571e+01, 9.8000e-01, 7.4200e+02, 2.5800e+03,
         1.8680e+03, 2.2800e+01, 1.0000e+00, 0.0000e+00, 0.0000e+00]]) tensor([[0.]])


In [25]:
class LoanDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.train = LoanDataset(train_x.values , train_y.values)
        self.test = LoanDataset(test_x.values , test_y.values)
        self.val = LoanDataset(val_x.values , val_y.values)
        
    def train_dataloader(self):
        return DataLoader(self.train,batch_size = 64 , shuffle = True)
    def test_dataloader(self):
        return DataLoader(self.test,batch_size = 32 , shuffle = False)
    def val_dataloader(self):
        return DataLoader(self.val,batch_size = 32 , shuffle = False)

In [26]:
loss_func  = torch.nn.BCEWithLogitsLoss()
sigmoid = nn.Sigmoid()
def metrics(y_pred,y):
    loss = loss_func(y_pred,y)
    y_pred = sigmoid(y_pred).detach()
    y_predictions = [1 if y_probs > 0.5 else 0 for y_probs in y_pred ]
    accuracy = accuracy_score(y,y_predictions) 
    precision = precision_score(y,y_predictions,zero_division=0,average='weighted')
    auc = roc_auc_score(y,y_pred)
    return accuracy,precision,auc,loss

In [27]:
class Loan_Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.initial_layer = nn.Sequential(
        nn.Linear(11,11),
        )
        self.hidden1 = nn.Sequential(
        nn.Linear(11,50),
        )

        self.hidden2 = nn.Sequential(
            nn.Linear(50,25)
        )
        self.hidden3 = nn.Sequential(
            nn.Linear(25,75)
        )
        self.hidden4 = nn.Sequential(
            nn.Linear(75,50)
        )
        self.hidden5 = nn.Sequential(
            nn.Linear(50,25)
        )
        self.hidden6 = nn.Sequential(
            nn.Linear(25,50)
        )
        self.hidden7 = nn.Sequential(
            nn.Linear(50,25)
        )
        self.hidden8 = nn.Sequential(
            nn.Linear(25,50)
        )
        self.hidden9 = nn.Sequential(
            nn.Linear(50,25)
        )
        self.hidden10 = nn.Sequential(
            nn.Linear(25,50)
        )
        self.hidden11 = nn.Sequential(
            nn.Linear(50,25)
        )
        self.final_layer = nn.Linear(25,1)

    
    def forward(self,x):
        out = self.initial_layer(x)
        out = nn.functional.relu(out)
        out = self.hidden1(out)
        out = nn.functional.relu(out)
        out = self.hidden2(out)
        ut = nn.functional.relu(out)
        out = self.hidden3(out)
        out = nn.functional.relu(out)
        out = self.hidden4(out)
        out = nn.functional.relu(out)
        out = self.hidden5(out)
        out = nn.functional.relu(out)
        out = self.hidden6(out)
        out = nn.functional.relu(out)
        out = self.hidden7(out)
        out = nn.functional.relu(out)
        out = self.hidden8(out)
        out = nn.functional.relu(out)
        out = self.hidden9(out)
        out = nn.functional.relu(out)
        out = self.hidden10(out)
        out = nn.functional.relu(out)
        out = self.hidden11(out)
        out = self.final_layer(out)
        return out
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters() , lr = 0.5e-3)
        sch = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
        return {
            "optimizer" : optimizer,
            "lr_scheduler" :{
                "scheduler" : sch,
                "monitor" :"train_loss"
            }
        }
    
    def validation_step(self,batch,batch_index):
        x,y = batch
        y_pred = self(x)
        accuracy,precision,auc,loss = metrics(y_pred,y)
        self.log('accuracy', accuracy)
        return loss
    def test_step(self,batch,batch_index):
        x,y = batch
        y_pred = self(x)
        accuracy,precision,auc,loss = metrics(y_pred,y)
        self.log('accuracy', accuracy)
        self.log('precision', precision)
        self.log('auc', auc)
        return loss
    def training_step(self,batch,batch_index):
        x,y = batch
        y_pred = self(x)
        loss = loss_func(y_pred,y)
        self.log("train_loss" , loss)
        return loss

In [28]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
checkpoint_callback = ModelCheckpoint(monitor = "accuracy", mode = "max")
model = Loan_Model()
module = LoanDataModule()
trainer = Trainer(max_epochs = 11)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [29]:
trainer.fit(model,module)


   | Name          | Type       | Params
----------------------------------------------
0  | initial_layer | Sequential | 132   
1  | hidden1       | Sequential | 600   
2  | hidden2       | Sequential | 1.3 K 
3  | hidden3       | Sequential | 1.9 K 
4  | hidden4       | Sequential | 3.8 K 
5  | hidden5       | Sequential | 1.3 K 
6  | hidden6       | Sequential | 1.3 K 
7  | hidden7       | Sequential | 1.3 K 
8  | hidden8       | Sequential | 1.3 K 
9  | hidden9       | Sequential | 1.3 K 
10 | hidden10      | Sequential | 1.3 K 
11 | hidden11      | Sequential | 1.3 K 
12 | final_layer   | Linear     | 26    
----------------------------------------------
16.8 K    Trainable params
0         Non-trainable params
16.8 K    Total params
0.067     Total estimated model params size (MB)


Epoch 10: 100%|██████████| 342/342 [00:14<00:00, 24.12it/s, loss=1.16, v_num=2]


1

In [30]:
trainer.test()

Testing: 100%|██████████| 171/171 [00:04<00:00, 39.68it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'accuracy': 0.6316849589347839, 'auc': 0.5, 'precision': 0.4057486057281494}
--------------------------------------------------------------------------------


[{'accuracy': 0.6316849589347839, 'precision': 0.4057486057281494, 'auc': 0.5}]